In [2]:
# This file is training code of Airfoil_DDPM.
# Author: Zhe Wen
# Date: 2025-5-22
# Copyright (c) Zhejiang University. All rights reserved.
# See LICENSE file in the project root for license information.

import torch
from torch import nn
from torch.nn import functional as F
from Airfoil_DDPM_pointcloud import Unet
from torch.utils.data import DataLoader,TensorDataset
from torch.utils.tensorboard import SummaryWriter
#from mlp_data import MyData
import pandas as pd
#import h5py
import csv
from math import sqrt
import numpy as np


import auxiliary.argument_parser as argument_parser
import auxiliary.Dataloader as Dataloader
import datetime
import logging
from pathlib import Path
import os

In [ ]:
def generate_gaussian_tensor(batch_size, dim, size):
    tensor = torch.empty(batch_size, dim, size)
    for i in range(batch_size):
        for j in range(dim):
            tensor[i, j] = torch.normal(mean=0.0, std=1.0, size=(size,))
    return tensor

def partial_load_state_dict(model, checkpoint):
    model_dict = model.state_dict()
    pretrained_dict = checkpoint['models']
    filtered_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and model_dict[k].shape == pretrained_dict[k].shape}
    model_dict.update(filtered_dict)
    model.load_state_dict(model_dict)
    return model

'''beta和alpha定义，用来混噪声合'''
def cal_alpha_bar(alpha, max_t):
    '''
    alpha : torch[max_t]
    sqrtalphabar, sqrt_1_m_alphabar : torch[max_t]
    '''
    sqrtalphabar=torch.empty(max_t)
    sqrt_1_m_alphabar=torch.empty(max_t)
    alphabar_temp=1
    for i in range(max_t):
        alphabar_temp=alphabar_temp*alpha[i]
        sqrtalphabar[i]=sqrt(alphabar_temp)
        sqrt_1_m_alphabar[i]=sqrt(1-alphabar_temp)
    return sqrtalphabar, sqrt_1_m_alphabar

alpha=1-torch.linspace(0.0001, 0.02, steps=1000)
sqrtalphabar,sqrt_1_m_alphabar=cal_alpha_bar(alpha, 1000)


def forward_propagation(model, loss, metric, input, context_1 = None, context_2 = None, time_step = 1000, device = 'cuda'):
    '''
    Input:
        input: Tensor [B, N, C]
        context : Tensor [(time_step-1)*B, 1, C] or None
    return: loss, metric
    '''
    noise_tensor = torch.randn(time_step, input.size(0), input.size(1), input.size(2)) #[time_step, B, N, C]
    input_tensor = input.unsqueeze(0) #[1,B,N,C]
    input_tensor = input_tensor.expand(time_step, *input_tensor.shape[1:])#广播#[time_step,B,N,C]
    # 将一维张量扩展到与 input_tensor 和 noise_tensor 相匹配的维度
    sqrtalphabar_expanded = sqrtalphabar.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand_as(input_tensor) #torch[time_step, 1 , 1, 1]->torch[time_step,B,N,C]
    sqrt_1_m_alphabar_expanded = sqrt_1_m_alphabar.unsqueeze(-1).unsqueeze(-1).unsqueeze(-1).expand_as(noise_tensor)#[time_step, B, N, C]
    # 使用向量化操作进行计算
    input_tensor = sqrtalphabar_expanded * input_tensor + sqrt_1_m_alphabar_expanded * noise_tensor #[time_step, B, N, C]
    #处理反向预测数据
    # 获取除了 t = 0 之外的切片
    noise_tensor_no_t0 = noise_tensor[1:]#[time_step-1, B, N, C]
    input_tensor_no_t0 = input_tensor[1:]#[time_step-1, B, N, C]
    # 沿着第一个维度展开
    noise_tensor_flattened = noise_tensor_no_t0.reshape(-1,noise_tensor_no_t0.shape[-2],noise_tensor_no_t0.shape[-1])#[(time_step-1)*B, N, C]
    input_tensor_flattened = input_tensor_no_t0.reshape(-1,input_tensor_no_t0.shape[-2],input_tensor_no_t0.shape[-1])#[(time_step-1)*B, N, C]
    # 获取 noise_tensor_flattened 的时间步总数
    N = noise_tensor_no_t0.shape[0] #time_step-1
    # 创建时间嵌入向量
    time_embedding = torch.arange(1, N + 1, dtype=torch.float32).unsqueeze(-1).expand(N, noise_tensor_no_t0.shape[1]) #[time_step-1 , 1*B]
    time_embedding = time_embedding.reshape(N*noise_tensor_no_t0.shape[1],-1) #[(time_step-1)*B , 1]

    time_embedding=time_embedding.to(device)
    input_tensor_flattened=input_tensor_flattened.to(device)
    noise_tensor_flattened=noise_tensor_flattened.to(device)
    print(input_tensor_flattened.shape)

    pred_noise = model(input_tensor_flattened, time_embedding, context_1=context_1, context_2=context_2)
    loss = loss(pred_noise, noise_tensor_flattened)
    metric = metric(pred_noise, noise_tensor_flattened)
    return loss, metric




def save_info(epoch, current_lr, loss, log_dir, type='step'):
    if type=='step':
        train_process=os.path.join(log_dir, 'step_info.csv')
    elif type=='epoch':
        train_process=os.path.join(log_dir, 'epoch_info.csv')

    with open(train_process, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([epoch, current_lr, loss])

In [4]:
    def log_string(str):
        logger.info(str)
        print(str)
    
    '''DEVICE'''
    device = 'cuda'

    '''CREATE DIR'''
    timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
    exp_dir = os.path.join('log')
    exp_dir = Path(exp_dir)
    # exp_dir = Path('E:/wenzhe/generate_3/Encoder-Decoder/log/')
    exp_dir.mkdir(exist_ok=True)
    exp_dir = exp_dir.joinpath('pointcloud_diffusion')
    exp_dir.mkdir(exist_ok=True)
    exp_dir = exp_dir.joinpath(timestr)
    exp_dir.mkdir(exist_ok=True)
    checkpoints_dir = exp_dir.joinpath('checkpoints/')
    checkpoints_dir.mkdir(exist_ok=True)
    log_dir = exp_dir.joinpath('logs/')
    log_dir.mkdir(exist_ok=True)

    '''LOG'''
    #args = parse_args()
    logger = logging.getLogger("Model")
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler = logging.FileHandler('%s/%s.txt' % (log_dir, 'pointcloud_diffusion'))
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    log_string('PARAMETER ...')
    # log_string(opt)

    train_process = os.path.join(log_dir, "train_process.csv")
    valid_process = os.path.join(log_dir, "valid_process.csv")

PARAMETER ...


In [ ]:
    '''Init'''
    df_model=Unet( num = [200, 100, 50, 100, 200], dim = [2, 2, 4, 2, 2], context_dim_1 = 3, context_dim_2 = 3, dropout = 0.).to(device)#
    # checkpoint = torch.load('E:/wenzhe/generation2/airfoil_diffusion/models3/DFmodel_context_c3.1.5_100.pth', map_location=torch.device(device),weights_only=True)   ### 加载神经网络模型
    # df_model = partial_load_state_dict(df_model, checkpoint)

    loss = nn.MSELoss().to(device)
    metric = nn.L1Loss().to(device)
    optim = torch.optim.Adam(df_model.parameters(), lr=0.0001)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=20, gamma=0.3, last_epoch=-1)
    time_step = 1000

In [ ]:
    '''Load Data'''
    data_path = os.path.join("../data/train_data_pointcloud.npz")
    data = np.load(data_path)
    loaded_pointcloud = np.transpose(data['pointcloud'],(0,2,1))  # 形状 [B, N, D]-> [B, D, N]
    loaded_ACC = data['ACC']                # 形状 [B, 6]

In [ ]:
    '''Dataloader'''
    # 划分数据集，创建Dataset
    len = loaded_pointcloud.shape[0]
    train_len = int(len*0.8)
    valid_len = int(len*0.1)
    train_dataset = Dataloader.PointCloudACCDataset(loaded_pointcloud[:train_len], loaded_ACC[:train_len])
    valid_dataset = Dataloader.PointCloudACCDataset(loaded_pointcloud[train_len:valid_len+train_len], loaded_ACC[train_len:valid_len+train_len])
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=1,#这不用注释了吧
        shuffle=True,# 打乱
        num_workers=4,# 多进程加载数
        pin_memory=True  # 加速GPU传输
    )
    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=32,#这不用注释了吧
        shuffle=True,# 打乱
        num_workers=4,# 多进程加载数
        pin_memory=True  # 加速GPU传输
    )

In [22]:
    train_step=0
    for batch in train_dataloader:
        pointcloud_batch = batch['pointcloud'].permute(0,2,1)  # 形状 [batch_size, N, 3]
        context_1 = batch['context_1']                # 形状 [batch_size, 3]
        context_2 = batch['context_2']                # 形状 [batch_size, 3]

        if context_1 is not None:
            context_1 = context_1.unsqueeze(1).to(device) #[B, 1, C]
            context_1 = context_1.unsqueeze(0) #[1, B, 1, C]
            #广播到时间维度上
            context_1 = context_1.expand(time_step-1, *context_1.shape[1:])#广播,但是少一次，因为t=0的时候不用 #[time_step-1, B, 1, C]
            context_1 = context_1.reshape(-1,context_1.shape[-2],context_1.shape[-1]) #[(time_step-1)*B, 1, C]
        
        if context_2 is not None:
            context_2 = context_2.unsqueeze(1).to(device)
            context_2 = context_2.unsqueeze(0)
            #广播到时间维度上
            context_2 = context_2.expand(time_step-1, *context_2.shape[1:])#广播,但是少一次，因为t=0的时候不用
            context_2 = context_2.reshape(-1,context_2.shape[-2],context_2.shape[-1])

        #第一次正向传播和反向传播
        step_loss, step_metric = forward_propagation(df_model, loss, metric, pointcloud_batch, context_1 = context_1, context_2 = context_2, time_step = time_step, device = 'cuda')
        break
            # optim.zero_grad()
            # step_loss.backward()
            # optim.step()
            # train_step+=1
            # current_lr = optim.param_groups[0]['lr']
            # with open(train_process, 'a', newline='') as csvfile:
            #     writer = csv.writer(csvfile)
            #     writer.writerow([epoch + 1,train_step,current_lr, step_loss.item(), step_metric.item()])

torch.Size([999, 1, 20])
